<img src='slides/wine_example.png'>

In [ ]:
import pandas as pd

# ItemIDとアイテム名の辞書
item_dict = {
    'I1': 'フランスワイン',
    'I2': 'イタリアワイン',
    'I3': 'スペインワイン',
    'I4': 'チリワイン',
    'I5': '国産ワイン'
}

# ワインの購買履歴のデータ
trans = [
    # TID: T1
    [1, 'I1'],
    [1, 'I2'],
    [1, 'I4'],
    [1, 'I5'],
    # TID: T2
    [2, 'I2'],
    [2, 'I3'],
    [2, 'I5'],
    # TID: T3
    [3, 'I1'],
    [3, 'I2'],
    [3, 'I4'],
    [3, 'I5'],
    # TID: T4
    [4, 'I1'],
    [4, 'I2'],
    [4, 'I3'],
    [4, 'I5'],
    # TID: T5
    [5, 'I1'],
    [5, 'I2'],
    [5, 'I3'],
    [5, 'I4'],
    [5, 'I5'],
    # TID: T6
    [6, 'I2'],
    [6, 'I3'],
    [6, 'I4']
]

# pandas DataFrame に変換、確認
df = pd.DataFrame(trans, columns=['TID', 'ItemID'])
df

### ItemID ごとの購買回数を確認

In [ ]:
# 'ItemID' 列の要素の頻度を表示
df.ItemID.value_counts()

<hr>
<img src='slides/12.png'>
<hr>

- support_count(): サポートカウントの計算
- support(): サポートの計算
- confidence(): コンフィデンスの計算

In [ ]:
from functools import reduce

# TIDを重複排除のため集合に変換
all_trans = set(df.TID)
# |D| : トランザクション数
all_trans_len = len(all_trans)

# 1つのitemについて、そのitemを含むTIDの集合を辞書で管理
count_dict = {}
# 全てのitemについて、そのitemを含むTIDの集合をtrans_setに追加
for item in df.ItemID.unique():
    count_dict[item] = set(df[df['ItemID'] == item].TID)

# サポートカウントの計算
# - v=1: verbose オプション
def support_count(item_set, v=0):
    trans_set = []
    # item_setの各itemについて、そのitemを含むTIDの集合をtrans_setに追加
    for item in item_set:
        trans_set.append(count_dict[item])
    # trans_set の確認用
    if (v >= 2):
        print('trans_set:', trans_set)
    # 全itemを含むTIDは、各itemを含むTIDの集合の積集合
    intersection = reduce(lambda a, x: a & x, trans_set, all_trans)
    # intersection の確認用
    if (v >= 1):
        print('intersection:', intersection)
    # サポートカウントは積集合の要素数
    return len(intersection)

# サポートカウントの確認
A = {'I1'}
B = {'I3'}
support_count(A | B, v=1)

<img src='slides/9.png'>

In [ ]:
# サポートの計算
def support(itemset, v=0):
    return support_count(itemset, v) / all_trans_len

# コンフィデンスの計算
def confidence(itemset_A, itemset_B, v=0):
    return support_count(itemset_A | itemset_B, v) / support_count(itemset_A, v)

# サポート、コンフィデンスの確認
print('{}⇒{}: サポート={:.3f}, コンフィデンス={:.3f}'.format(
    A, B, support(A | B, v=1), confidence(A, B, v=1)))

<hr>

## 相関ルールのマイニング手順

相関ルールのマイニングは以下の2つのステップからなる

1. Aprioriアルゴリズム
   - すべての頻出アイテムセットを抽出
2. 相関ルールの生成
   - 頻出アイテムセットから強いルールを生成

<hr>

## 1. Aprioriアルゴリズム

- generate_candidates(): 𝑘個の要素を持つ頻出アイテムセットの候補 𝐶_𝑘 を作成
- apriori(): すべての頻出アイテムセットを抽出

In [ ]:
from itertools import combinations

# Generate_Candidates(L_{k-1}, k, min_sup_count)
def generate_candidates(L, k, min_sup_count, v=0):
    Ck = []
    for l1 in L:
        for l2 in L:
            # listのインデックスは 0 始まり
            # - l1[0:n]    　…l1[0…(n-1)] を返す
            # - l1[0:k - 2]　…(k-2)が0の時, 0~-1の範囲はないので空のリストになる
            if l1[0:k - 2] == l2[0:k - 2] and l1[k - 2] < l2[k - 2]:
                # l1とl2の和集合
                c = sorted(list(set(l1) | set(l2)))
                if (v >= 1):
                    print('k={}: {} | {} = {}'.format(k, l1, l2, c))
                # cのk-1個のアイテムからなる部分集合について
                for i in combinations(c, k - 1):
                    # サポートカウントを算出
                    sc = support_count(i)
                    if (v >= 2):
                        # iはタプルのため、('I1',) のように出力される
                        print('  support_count({})={}'.format(i, sc))
                    # サポートカウントが min_sup_count より小さいか
                    if sc < min_sup_count:
                        # 頻出でないならば追加しない
                        break
                else:
                    # さもなければ、Ckに追加
                    # for else 構文
                    # - for ループを break で抜けた時には else は実行されない
                    # - 頻出でないならば追加しないため、break すると追加されない
                    Ck.append(c)
    return Ck

# Apriori アルゴリズムの処理
def apriori():
    # 全てのアイテム
    all_items = sorted(list(set(df.ItemID)))

    # 頻出アイテムセットの集合を格納するdict型変数
    L = {}

    # L1 ← 1つのアイテムからなる頻出アイテムセットの集合
    L[1] = [[i] for i in all_items if support_count({i}) >= min_sup_count]
    # print('L1 = ', L[1])

    # k ← 2
    k = 2

    while L[k - 1] != []:
        Lk = []
        Ck = generate_candidates(L[k - 1], k, min_sup_count)
        for c in Ck:
            # print('{} {}'.format(c, support_count(c)))
            if support_count(c) >= min_sup_count:
                Lk.append(c)
        L[k] = Lk
        # print('L{} = {}'.format(k, L[k]))
        k = k + 1
    
    # 和集合
    L_union = []
    for ln in L.values():
        for l in ln:
            L_union.append(l)

    return L_union

### Aprioriアルゴリズムの実行

In [ ]:
# 最小サポート: 0.5, 最小サポートカウント
min_sup = 0.5
min_sup_count = min_sup * all_trans_len

# Aprioriアルゴリズムの実行
L_union = apriori()

# 得られた和集合の確認
L_union

<hr>

## 2. 相関ルールの生成

<img src='slides/41.png'>

In [ ]:
l = ['I2', 'I3', 'I5']

# 全ての真部分集合 all_s のリストを初期化
all_s = []

# 全ての真部分集合 all_s の作成
for n in range(1, len(l)):
    for i in combinations(l, n):
        all_s.append(list(i))
# all_sの確認
print('all_s = {}'.format(all_s))

# lのサポートカウント
l_support_count = support_count(l, v=0)

# 各真部分集合についてコンフィデンスを計算
for s in all_s:
    # コンフィデンス(s ⇒ (l-s))
    s_support_count = support_count(s, v=0)
    c = l_support_count / s_support_count
    print('{} {}/{}={}'.format(s, l_support_count, s_support_count, c))

<img src='slides/42.png'>

In [ ]:
# 最小コンフィデンス
min_confidence = 0.7

# 相関ルールの初期化
assoc_rule = []

# 各真部分集合についてコンフィデンスを計算
for s in all_s:
    # コンフィデンス(s ⇒ (l-s))
    s_support_count = support_count(s, v=0)
    c = l_support_count / s_support_count
    print('{} {}/{}={}'.format(s, l_support_count, s_support_count, c))
    # 最小コンフィデンス以上のルールを相関ルールに追加
    if c >= min_confidence:
        #                  s ⇒ (l-s) をリストで表す
        assoc_rule.append([s, [i for i in l if i not in s]])

# 相関ルールの確認
assoc_rule